In [1]:
print("hola gordo")

hola gordo


In [11]:
import pandas as pd
from scipy import spatial


In [2]:
PATH = r'C:\Users\brian\OneDrive\Escritorio\PROJECTS\UPV\sistemas_recomendacion\data'
genre_names = [
    'movie_id','unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 
    'War', 'Western', 'movie_name']
df_items = pd.read_csv(PATH+'\items.txt',sep="	", encoding='latin-1',header=None, names= genre_names)
df_genre = pd.read_csv(PATH+'\genre.txt',sep="	", encoding='latin-1',header=None)
df_calif = pd.read_csv(PATH+r'\u1_base.txt',sep="	", encoding='latin-1',header=None,names=["users","movie_id","rating"])
df_users = pd.read_csv(PATH+r'\users.txt',sep="	", encoding='latin-1',header=None, names=["users","age","gender","ocuppation"])

In [3]:
df_concat = df_users.merge(df_calif, left_on='users', right_on='users')
df_full = df_concat.merge(df_items, left_on='movie_id', right_on='movie_id')

In [4]:
#crear vector de preferencias
df_pref = pd.DataFrame([])
generos_lista = ['unknown','Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
df_pref[generos_lista] = df_full[generos_lista].mul(df_full['rating'], axis= 0 )
df_pref['users']  = df_full["users"]
df_pref = df_pref.groupby('users').mean()

In [36]:
df_full

,users,age,gender,ocuppation,movie_id,rating,unknown,Action,Adventure,Animation,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_name
0,1,24,M,technician,1,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
1,2,53,F,other,1,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
2,6,42,M,executive,1,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
3,13,47,M,educator,1,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
4,16,21,M,entertainment,1,5,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Toy Story (1995)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,863,17,M,student,1679,3,0,0,0,0,...,0,0,0,0,1,0,1,0,0,B. Monkey (1998)
79996,863,17,M,student,1680,2,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Sliding Doors (1998)
79997,871,31,M,executive,907,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Vermin (1998)
79998,896,28,M,writer,1681,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,You So Crazy (1994)


### sistema coolabora

In [35]:
df_pref

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
users,,,,,,,,,,,,,,,,,,,
1,0.0,0.703704,0.311111,0.185185,0.237037,1.288889,0.274074,0.177778,1.651852,0.022222,0.037037,0.096296,0.096296,0.103704,0.681481,0.585185,0.659259,0.318519,0.029630
2,0.0,0.425000,0.100000,0.100000,0.200000,0.825000,0.600000,0.000000,2.275000,0.000000,0.125000,0.075000,0.075000,0.200000,1.325000,0.075000,0.825000,0.150000,0.000000
3,0.0,0.607143,0.357143,0.000000,0.000000,0.714286,0.428571,0.178571,1.392857,0.000000,0.071429,0.107143,0.142857,0.535714,0.321429,0.607143,0.785714,0.321429,0.000000
4,0.0,1.571429,0.642857,0.000000,0.000000,0.714286,1.000000,0.357143,1.071429,0.000000,0.000000,0.000000,0.357143,0.928571,0.214286,0.785714,1.714286,0.285714,0.000000
5,0.0,1.131868,0.758242,0.395604,0.362637,1.395604,0.186813,0.000000,0.373626,0.000000,0.000000,0.428571,0.285714,0.076923,0.252747,0.824176,0.230769,0.296703,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,1.489796,0.714286,0.081633,0.081633,1.387755,0.265306,0.000000,1.673469,0.081633,0.000000,0.061224,0.081633,0.081633,0.979592,0.673469,1.000000,0.306122,0.000000
940,0.0,0.738318,0.336449,0.084112,0.196262,1.383178,0.280374,0.000000,1.616822,0.000000,0.112150,0.112150,0.205607,0.121495,0.794393,0.542056,0.626168,0.457944,0.000000
941,0.0,1.727273,1.227273,0.636364,0.409091,1.318182,0.136364,0.000000,0.954545,0.000000,0.000000,0.000000,0.181818,0.227273,0.227273,1.409091,1.272727,0.227273,0.000000


In [12]:
# sistema por contenido compara un usuario contra todos los suarios
#hacer pearson
#regresar n mejores usuarios
# definir una formula para buscar afinidad entre las peliculas que aparezcan entre los usuarios.
# tendriamos que encontrar los n mejroes usuarios. Listar las peliculas de esos mejores nsuarios
#crear un ratio de afinidad para cada pelicula vista por los n usuarios mas cercanos tomando en cuenta el ratio de afinidad entre usuarios y la puntuacion 
movie_a = df_pref.iloc[0].to_list()
movie_b = df_pref.iloc[10].to_list()
spatial.distance.cosine(movie_a, movie_b)

0.01763214048272399

In [28]:
df_full[df_full['users']==1][['rating','movie_name']].sort_values(by='rating', ascending=False)

,rating,movie_name
0,5,Toy Story (1995)
7250,5,Maya Lin: A Strong Clear Vision (1994)
9826,5,Monty Python and the Holy Grail (1974)
9718,5,Manon of the Spring (Manon des sources) (1986)
9663,5,Jean de Florette (1986)
...,...,...
16272,1,Air Bud (1997)
16306,1,Steel (1997)
14440,1,Batman Returns (1992)
8640,1,D3: The Mighty Ducks (1996)


In [38]:
# ssitema colaborativo: comparo con la informacion de un usuario checo las peliculas que  mas le han gustado y le recomiendo con base en esa pelicula
df_full[df_full['users']==1][['rating','movie_name']].sort_values(by='rating', ascending=False)

,rating,movie_name
0,5,Toy Story (1995)
7250,5,Maya Lin: A Strong Clear Vision (1994)
9826,5,Monty Python and the Holy Grail (1974)
9718,5,Manon of the Spring (Manon des sources) (1986)
9663,5,Jean de Florette (1986)
...,...,...
16272,1,Air Bud (1997)
16306,1,Steel (1997)
14440,1,Batman Returns (1992)
8640,1,D3: The Mighty Ducks (1996)


### basado contenido

pedir informacion  a un usuario nuevo, este tiene que ratear los generos. esta puntuacion crea un vector de preferencias para cada genero, tomar las n mejores calificaciones de cada genero y volver 0 el resto.

una vez que volvemos 0 el resto ver como vamos hacer la comparacion del vector de preferencias actual, con el resto
quitando las que ya vio 

In [46]:
df_full.groupby('movie_id')[generos_lista].mean()

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [45]:
df_full.groupby('movie_id')[generos_lista].mean()
movie_a = df_pref.iloc[0].to_list()
movie_b = df_pref.iloc[10].to_list()
spatial.distance.cosine(movie_a, movie_b)

0.01763214048272399

In [29]:
df_full.columns

Index(['users', 'age', 'gender', 'ocuppation', 'movie_id', 'rating', 'unknown',
       'Action', 'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'movie_name'],
      dtype='object')

In [30]:
#sistema demografico, recomiendo a traves de grupos 
lista_preferencias = ['age', 'gender', 'ocuppation']
df_2 = df_full[lista_preferencias]

In [33]:
df_2_num = pd.get_dummies(df_2[lista_preferencias], prefix='category')


In [37]:
df_2_num

,age,category_F,category_M,category_administrator,category_artist,category_doctor,category_educator,category_engineer,category_entertainment,category_executive,...,category_marketing,category_none,category_other,category_programmer,category_retired,category_salesman,category_scientist,category_student,category_technician,category_writer
0,24,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,53,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,42,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,47,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,17,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
79996,17,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
79997,31,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
79998,28,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
df_2

,age,gender,ocuppation
0,24,M,technician
1,53,F,other
2,42,M,executive
3,47,M,educator
4,21,M,entertainment
...,...,...,...
79995,17,M,student
79996,17,M,student
79997,31,M,executive
79998,28,M,writer


In [ ]:
from lightgbm import LGBMClassifier


num_folds = 5
kf = KFold(n_splits = num_folds, random_state = 42)
error = 0
models = []
for i, (train_index, val_index) in enumerate(kf.split(train)):
    if i + 1 < num_folds:
        continue
    print(train_index.max(), val_index.min())
    train_X = train[train_index]
    val_X = train[val_index]
    train_y = target[train_index]
    val_y = target[val_index]
    lgb_train = lgb.Dataset(train_X, train_y > 0)
    lgb_eval = lgb.Dataset(val_X, val_y > 0)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
             
            }
    gbm_class = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
    
    lgb_train = lgb.Dataset(train_X[train_y > 0], train_y[train_y > 0])
    lgb_eval = lgb.Dataset(val_X[val_y > 0] , val_y[val_y > 0])
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
                         }
    gbm_regress = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
#     models.append(gbm)

    y_pred = (gbm_class.predict(val_X, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(val_X, num_iteration=gbm_regress.best_iteration))
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    print(np.sqrt(mean_squared_error(y_pred, (val_y))))
    break
print(error)

In [51]:
df_concatenado = pd.concat([df_users.set_index('users'), df_calif.set_index('users')], axis=1)
df_concateated = pd.concat([df_concatenado.set_index('movie_id'), df_items.set_index('movie_id')], axis=1)
# Si quieres resetear el índice después de la concatenación:
df_concatenado.reset_index(inplace=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [45]:
# Specify the columns to encode
columns_to_encode = ['gender','ocuppation']

# Perform one-hot encoding on selected columns
df_encoded = pd.get_dummies(df_concatenado
, columns=columns_to_encode)

In [46]:
df_encoded

,level_0,index,users,age,movie_id,rating,gender_F,gender_M,ocuppation_administrator,ocuppation_artist,...,ocuppation_marketing,ocuppation_none,ocuppation_other,ocuppation_programmer,ocuppation_retired,ocuppation_salesman,ocuppation_scientist,ocuppation_student,ocuppation_technician,ocuppation_writer
0,0,0,1,24,1,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,1,24,2,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,2,1,24,3,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,3,1,24,4,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,4,1,24,5,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,79995,79995,943,22,1067,2,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79996,79996,79996,943,22,1074,4,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79997,79997,79997,943,22,1188,3,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
79998,79998,79998,943,22,1228,3,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
